In [117]:
# First we import the libraries
# Please make sure you have the required libraries.
# You can use pip or conda to get them.

import numpy as np

from qucumber.nn_states import PositiveWaveFunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

import torch

# set random seed on cpu but not gpu, as gpu is not used
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [183]:
# Now we get the training data
# Training data is given in the data/nY=8/ directory for each delta
# in the '...samples.csv' files.
# So the delta needs to be specified below

#-------------------- Specify delta and nY ---------------------#
delta = "1.14" 
nY = "8" # nY refers to the number of atoms in the array
# For the nY = 8 system case:
# delta is any of 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20
# For nY > 8:
# delta is any of 1.00, 1.04, 1.08, 1.12, 1.16, 1.20, 1.28
#---------------------------------------------------------------#

train_path = "data/nY="+nY+"/δ="+delta+"_samples.csv"
train_data = data.load_data(train_path)

# The training data is stored in train_data[0]
# and the dimension of the data is:
train_data[0].shape

torch.Size([10000, 8])

In [184]:
# So now we have the training data. Our goal is to train a
# Restricted Boltzman Machine using this training data. We use
# QuCumber to create an instance of an RBM. First we need to specify the 
# number of visible nodes (nv) and number of higgen nodes (nh). 
# As we have an array of 8 atoms, we have 8 inputs and so nv = 8
nv = train_data[0].shape[-1]

# Number of hidden nodes of an RBM is a hyperparameter which depends on the 
# data we are using and which can be varied to get optimal result. 
# For delta=1.14, nh was varied from 4 to 12 and nh = 12 gave quite good results.
# Hence, we set nh = 12.
# Note that for different delta, nh=12 may not give good results.
nh = 12

# Finally we create an RBM with nv visible nodes and nh  hidden nodes.
nn_state = PositiveWaveFunction(num_visible=nv, num_hidden=nh, gpu=False)

In [185]:
# Below we have more hyperparameters of the RBM model. Like the number of 
# hidden nodes, the parameters below can be varied to get optimal results.
# The following hyperparameters seemed to work quite well. Note that when
# you have different data, then different values of the hyperparameter will give
# better solutions. 
# Further description of the parameters (if you are curious) is in:
# https://qucumber.readthedocs.io/en/stable/quantum_states.html?highlight=fit#qucumber.nn_states.PositiveWaveFunction.fit

epochs = 500
pbs = 100
nbs = pbs
lr = 0.001 # For nY > 8 systems lr = 0.0065 can be used as it works quite well.
k = 10

# Now we train our RBM using the above parameters. 
# Note that the training process will take 2 - 3 minutes

nn_state.fit(
    train_data[0],
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    time=True,
)

Total time elapsed during training: 121.132 s


In [186]:
# After training is complete, we save the parameters of the trained RBM below
# as rydberg_data.pt in the output directory.
nn_state.save("output/rydberg_data.pt")
torch.load("output/rydberg_data.pt")
# Below we have the parameters  (weights and biases) of the trained RBM:

{'rbm_am': OrderedDict([('weights',
               tensor([[ 0.0436,  0.1365, -0.1789, -0.2058,  0.0070, -0.2799,  0.0170, -0.0827],
                       [ 0.5144, -0.4623,  0.2377,  0.0121, -0.3839, -0.3526,  0.2013, -0.4161],
                       [ 0.1809, -1.1153,  0.9791, -1.1769,  1.1068, -0.9989,  0.5834, -0.6266],
                       [-0.9534,  0.8799, -0.9635,  0.8807, -0.9633,  0.5620, -0.7005,  0.7425],
                       [-0.7777,  0.6887, -0.5053,  0.4842, -0.8278,  0.4021, -1.1933,  0.9725],
                       [-0.1604,  0.1566, -0.7861,  0.0782,  0.2232, -0.3297,  0.3641, -0.7745],
                       [-0.9716,  0.7356, -1.1741,  0.6283, -1.0942,  1.0014, -0.7571,  0.4562],
                       [-0.1474, -0.2230, -0.0836,  0.6210, -0.9229,  0.6951, -0.2905, -0.1504],
                       [ 0.4037, -0.9583,  0.5166, -0.3403, -0.0837, -0.2000,  0.4310, -0.5927],
                       [ 0.0930, -0.5438,  0.2563, -0.5043,  0.3958, -0.6337,  0.3522, -0.3

In [187]:
# Now we have our trained RBM. Here we reconstruct the wavefunction by sampling from the RBM. 
# Let us see step by step how it works -

# The first step is to sample from our trained RBM using QuCumber.
# We then get num_samples samples which is stored in the variable 'samples'
num_samples = 10000
samples = nn_state.sample(num_samples = num_samples , k = 10)

# Now we print out the samples in 'output/reconstructedSample.txt'
sampleList = samples.tolist() # convert to list for convenience
sampleList
with open('output/reconstructedSample.txt', 'w') as fp:
    for item in sampleList:
        fp.write("%s\n" % item)

In [188]:
## -------------------------------------------------------------------- ##
## This block of code computes amplitudes. For nY > 8, it is not necessary
## to run this block as computation takes a lot of time.
## -------------------------------------------------------------------- ##

# From the samples we obtain the amplitudes of each of the basis states.
# Note that here we have an 8 atom array. Hence, there are 2^8 basis states
# which are: |00000000>, |00000001>, ..., |11111111>

# Now say X is any of the states from {|00000000>, |00000001>, ..., |11111111>}
# Now, the amplitude of a state X is computed by taking the
# square root of the number of states X present in 
# our produced sample in 'output/reconstructedSample.txt'.

# Therefore the reconstructed sample is like: 
#    |psi> = amplitudeList[0] |00000000> + amplitudeList[1] |00000001> + ... + amplitudeList[255] |11111111>  
# where amplitudeList is defined below.

# The following functions below finds the amplitudes in the way described above, stores the 
# amplitudes in amplitudeList and then also 
# prints out the amplitudes in 'output/reconstructedSample.txt'

def amplitude(sample):
    return np.sqrt(sampleList.count(sample)/num_samples)

def getBinaryString(i):
    sites = nv # nv = 8
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    tempStr = getbinary(i, sites)
    return tempStr

# Python code to convert string to list character-wise
def ConvertToList(string):
    list1=[]
    list1[:0]=string
    return list1

amplitudeList = []
def getAmplitude(sample):
    for i in range (2**nv):
        binaryString = getBinaryString(i)
        strList = ConvertToList(binaryString)
        tempStr = str(amplitude(list(map(int, strList))))
        amplitudeList.append(tempStr)
    return 0

getAmplitude(sampleList)
with open('output/reconstructedStateAmplitudes.txt', 'w') as fp:
    for item in amplitudeList:
        fp.write("%s\n" % item)

In [189]:
# So now we have obtained the reconstructed wavefunction.
# Now in the 'data' directory for each delta we have 1_pt_fun and 2_pt_fn
# 1_pt_fn is the one point function and 
# 2_pt_fn is the 2 point function.

# 1_pt_fn is the average occupation of each site. Note that
# there are 8 sites as there are 8 atoms. Occupation of each site
# refers to the proportion of atoms in the up state (state 1) 
# in each state.

# Now we get average occupation of each site by finding the 
# number of atoms in the 1 state in each site and then
# by dividing by the number of the produced samples 
occupation = [0]*nv
for i in range(len(sampleList)):
    j = 0
    for j in range(nv):
        occupation[j] = sampleList[i][j] + occupation[j]
for i in range(nv):
    occupation[i] = occupation[i]/len(sampleList)

# Thus the average occupation per site of each site for the reconstructed state is given below
# This data can be compared to data/nY=8/δ=delta_1_pt_fn.csv for the relevant delta

# (Note: The list in data/nY=8/δ=delta_1_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the data in 'occupation' and 'data/nY=8/δ=delta_1_pt_fn.csv' are expected to be close (which they are)
occupation

[0.399, 0.3969, 0.4121, 0.4132, 0.3944, 0.4195, 0.3856, 0.4128]

In [190]:
# Now we look at the 2 point function
# 2 point function is the covariance matrix for occupations.
# It is found by 2_pt_fn = outerProdT1 - OuterProdT2

# outerProdT1 is the average of the outer product of the spin vectors with themselves
# So we first find the sum of the outerproducts of the spin vectors with themselves.
# Then we take the average
outerProdT1 = np.zeros((nv,nv))
for i in range(len(sampleList)):
    outerProdT1 = outerProdT1 + np.outer(sampleList[i], sampleList[i])
outerProdT1 = outerProdT1/len(sampleList)

# outerProd2 is simply the outer product of the occupation vector
# with itself. Note that we already found the occupation vector for the 
# one point function

outerProdT2 = np.outer(occupation,occupation)
outerProdT2

#Hence we have:
TwoPointFunc = outerProdT1 - outerProdT2

# The TwoPointFunction data for the reconstructed state is given below.
# This matrix can be compared to data/nY=8/δ=delta_2_pt_fn.csv for the relevant delta

# (Note: The matrix in data/nY=8/δ=delta_2_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the matrix in 'TwoFuncPoint' and 'data/nY=8/δ=delta_2_pt_fn.csv' are expected 
# to be nearly equal (which they are)
TwoPointFunc

array([[ 0.239799  , -0.1195631 ,  0.1284721 , -0.1196668 ,  0.1225344 ,
        -0.1231805 ,  0.1236456 , -0.1207072 ],
       [-0.1195631 ,  0.23937039, -0.12536249,  0.12710092, -0.12193736,
         0.12710045, -0.11804464,  0.12655968],
       [ 0.1284721 , -0.12536249,  0.24227359, -0.12507972,  0.12896776,
        -0.12637595,  0.12369424, -0.12291488],
       [-0.1196668 ,  0.12710092, -0.12507972,  0.24246576, -0.12536608,
         0.1329626 , -0.11992992,  0.12683104],
       [ 0.1225344 , -0.12193736,  0.12896776, -0.12536608,  0.23884864,
        -0.1266508 ,  0.12311936, -0.12180832],
       [-0.1231805 ,  0.12710045, -0.12637595,  0.1329626 , -0.1266508 ,
         0.24351975, -0.1207592 ,  0.1309304 ],
       [ 0.1236456 , -0.11804464,  0.12369424, -0.11992992,  0.12311936,
        -0.1207592 ,  0.23691264, -0.12347568],
       [-0.1207072 ,  0.12655968, -0.12291488,  0.12683104, -0.12180832,
         0.1309304 , -0.12347568,  0.24239616]])